In [ ]:
!pip install --upgrade huggingface_hub
!pip install --upgrade fireworks-ai
#!pip install telebot
!pip install python-telegram-bot --upgrade
!pip install pyTelegramBotAPI

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.1/111.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 673.5/673.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.5/270.5 kB 18.3 MB/s eta 0:00:00


In [ ]:
import telebot
import pandas as pd
import random
from huggingface_hub import InferenceClient
import logging
from collections import deque
from telebot import types
from PIL import Image
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

logging.basicConfig(level=logging.INFO)

In [ ]:
!git clone https://github.com/frdaria/elfo_bot.git

Cloning into 'elfo_bot'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 25 (delta 1), reused 5 (delta 1), pack-reused 19 (from 1)
Receiving objects: 100% (25/25), 89.99 MiB | 16.67 MiB/s, done.
Resolving deltas: 100% (5/5), done.
Updating files: 100% (7/7), done.


In [ ]:
df = pd.read_csv('elfo_bot/prompts_fandom.csv', on_bad_lines='skip')
chunks = pd.read_csv('elfo_bot/chunks_kinopoisk.csv', on_bad_lines='skip')
vectors = np.load('/content/elfo_bot/bg3_m3_kinopoisk_chunks.npy')
chunks_litres = pd.read_csv('elfo_bot/chunks_litres2.csv', on_bad_lines='skip')
vectors_litres = np.load('/content/elfo_bot/bg3_m3_litres_chunks2.npy')

In [ ]:
client_gemma = InferenceClient(
    provider="hf-inference",
    api_key=api_key,)
# client_picture = InferenceClient(
#     provider="fal-ai",
#     api_key=api_key,)
# client_mistral = InferenceClient(
#     provider="fireworks-ai",
#     api_key=api_key,
# )

In [ ]:
#embedding model
model = SentenceTransformer("BAAI/bge-m3")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
prompt_system_dialogue = '''Ты - волшебный эльф по имени Элфо! Ты - добрый писатель и учитель, который хочет помочь ученикам лучше ориентироваться в классической литературе. Ты пишешь приличный, дружелюбный и захватывающий контент на русском языке. Иногда ты используешь эмодзи. Твои тексты просты для чтения, полны диалогов, юмора и шуток! Ты пишешь фанфики по мотивам русских классических книг. Ты используешь разных персонажей классических книг и даешь пользователю информацию про него. Когда пользователь дает тебе тему истории, ты начинаешь со слов "Однажды..." и заканчиваешь словами "Продолжим читать?". Если пользователь задает тебе вопрос, не связанный с фанфиками по классическим произведениям, ты отвечаешь: 'Я просто Элфо! Какую историю ты хочешь почитать?'''

# def dialogue_model(user_question: str, context: list) -> str:
#     messages = [
#                 {
#                   "role": "system",
#                   "content": prompt_system_dialogue
#                 },
#                ]
#     if context:
#          messages.extend(context)

#     question = [{'role':'user',
#                  'content': user_question}]
#     messages.extend(question)
#     completion = client_mistral.chat.completions.create(
#       model="mistralai/Mistral-Small-24B-Instruct-2501",
# 	    messages=messages,
#   	  max_tokens=200,
#     )
#     return completion.choices[0].message.content


def update_context(context, user_m, answer):
    new_data = [{'role': 'user', 'content': user_m},
                {'role': 'system', 'content': answer}]
    context.extend(new_data)
    return context

def get_random_prompt():
    prompts = [
        "Жил-был дракон, который любил чай...",
        "Однажды в далёкой галактике...",
        "В маленькой деревне жил волшебник, который боялся магии..."
    ]
    return random.choice(prompts)

def get_fandom_prompt(df, fandom):
    if fandom in df.columns:
        prompts = df[fandom].dropna().tolist()
        if prompts:
            return random.choice(prompts)
        else:
            return "Для этого фандома нет промптов."
    else:
        return "Фандом не найден."


def process_dialogue_step(user_message, context, search_res):
        try:
            answer = gemma_generate(user_message, list(context), search_res=search_res if search_res else "")
        except Exception as e:
            logging.error(f"Ошибка в gemma_generate: {e}")
            return None, None
        if not answer:
            logging.error("Пустой ответ от gemma_generate")
            return None, None
        context = update_context(context, user_message, answer)
        logging.info(f"Updated context: {context}")
        return answer, context

#sys_prompt_gemma = """Ты волшебный Эльф, который пишет волшебные фанфики на русском языке. Твоя задача: 1) по заданному контексту написать историю. 2) Написать промпт для иллюстрации к этой истории. Используй шаблон: 'История:...Иллюстрация:...'Всегда следуй этому шаблону!!!"""
prompt_for_search_results = """Прочитай результаты поиска по запросу пользователя. Пользователь еще не видел этих результатов. Предложи мне использовать это произведение как основу фанфика."""

def gemma_generate(user_question: str, context: list, search_res) -> str:
    messages=[{
                "role": "system",
                "content": [{"type": "text", "text": prompt_system_dialogue}]
              }]
    if context:
         messages.extend(context)

    question = [{"role":"user",
                 "content": user_question}]
    messages.extend(question)

    if search_res:
      extra_info_from_retriever = [{"role":"assistant",
             "content": f'{prompt_for_search_results}. Результат поиска: {search_res}'}]

      messages.extend(extra_info_from_retriever)
      print(f"[DEBUG] messages added")

    completion = client_gemma.chat.completions.create(
                model="google/gemma-3-27b-it",
                max_tokens=200,
                messages = messages)

    gemma_answer = completion.choices[0].message.content.replace('\n\n', '').replace('##', '')
    return gemma_answer

def gemma_create_prompt(story) -> str:
    messages=[{
                "role": "user",
                "content": [{"type": "text", "text": f'Read the text and create a SIMPLE and SHORT prompt to illustrate the plot of the story. The prompt must be in English. Do not give any comments, your ouput must consist of the promplt only!!! Story: {story}'}]
              }]
    completion = client_gemma.chat.completions.create(
                model="google/gemma-3-27b-it",
                max_tokens=50,
                messages = messages)

    description = completion.choices[0].message.content.replace('\n\n', '').replace('##', '')
    return description

def picture_generate(picture_prompt):
    # output is a PIL.Image object
    image = client_picture.text_to_image(f'mystical style inspired by old fairy tales books, flat design. {picture_prompt}',
    model="stabilityai/stable-diffusion-3.5-large",)
    return image

def find_similar(query, data, vectors, model, top_k=1):
    query_vec = model.encode(query)
    similarities = cosine_similarity([query_vec], vectors)[0]
    top_indices = similarities.argsort()[-top_k:][::-1]
    results = []
    for idx in top_indices:
        title = data.iloc[idx]['title']
        description = data.iloc[idx]['description']
        results.append(f'Название: {title}, Описание: {description}')
    return results

In [ ]:
# results = find_similar("драконы", chunks_litres, vectors_litres, model, top_k=1)
# print(results)

In [ ]:
# Ограничиваем контекст последними n сообщениями
MAX_CONTEXT_LENGTH = 7
context_q = deque(maxlen=MAX_CONTEXT_LENGTH)
# dialog_check = 0
# answer = ''
user_states = {}


# @bot.message_handler(commands=["очистить"])
# def clear_context(message):
#     global context_q
#     context_q.clear()  # Очищаем контекст
#     bot.reply_to(message, "Контекст очищен. Начнём с чистого листа!")

fandoms = ["🌹💌 Евгений Онегин", "🧙✨ Властелин Колец", "👑⚔️ Игра престолов", "🔥🐲 Dragon Age", "🦹🏻❤️ Алые сердца Корё"]

def create_picture_keyboard():
    keyboard = types.ReplyKeyboardMarkup(row_width=2, resize_keyboard=True)
    button_fandom = types.KeyboardButton("Изобразить")
    keyboard.add(button_fandom)
    return keyboard

# Создаем клавиатуру
def create_keyboard():
    keyboard = types.ReplyKeyboardMarkup(row_width=2, resize_keyboard=True)
    button_fandom = types.KeyboardButton("📚 Выбрать фандом")
    button_interactive_fanfic = types.KeyboardButton("Интерактивный фанфик")
    #button_guess_book = types.KeyboardButton("Угадай книгу по сюжету")
    keyboard.add(button_fandom, button_interactive_fanfic)
    return keyboard

def create_options_for_interactive_acts():
    keyboard = types.ReplyKeyboardMarkup(row_width=2, resize_keyboard=True)
    button_sem_search = types.KeyboardButton("Включить поиск")  # ищем информацию, если модель не справляется
    keyboard.add(button_sem_search)
    return keyboard


def create_fandom_keyboard():
    keyboard = types.ReplyKeyboardMarkup(row_width=2, resize_keyboard=True)
    button_eo = types.KeyboardButton("🌹💌 Евгений Онегин")
    button_lotr = types.KeyboardButton("🧙✨ Властелин Колец")
    button_ip = types.KeyboardButton("👑⚔️ Игра престолов")
    button_da = types.KeyboardButton("🔥🐲 Dragon Age")
    button_core = types.KeyboardButton("🦹🏻❤️ Алые сердца Корё")
    keyboard.add(button_eo, button_lotr, button_ip, button_da, button_core)
    return keyboard


@bot.message_handler(func=lambda message: message.text == "Изобразить")
def handle_picture_request(message):
    global context_q
    if context_q:
        last_answer = context_q[-1]['content']
        if context_q[-1]['role'] == 'system':
            pic_prompt = gemma_create_prompt(last_answer)
        else:
            pic_prompt = context_q[-2]['content'] if len(context_q) > 1 else ""
    else:
        pic_prompt = ""
    try:
        image = picture_generate(pic_prompt)
        image.save('generated_image.jpg')
        bot.send_photo(message.chat.id, open('generated_image.jpg', 'rb'))
        bot.send_message(message.chat.id, "Выберите фандом:", reply_markup=create_keyboard())
    except Exception as e:
        print(f"Ошибка генерации: {e}")
        bot.reply_to(message, "Не удалось создать изображение 😔")


@bot.message_handler(func=lambda message: message.text == "📚 Выбрать фандом")
def choose_fandom(message):
    bot.reply_to(message, "Выбери фандом:", reply_markup=create_fandom_keyboard())

@bot.message_handler(func=lambda message: message.text == "Интерактивный фанфик")
def handle_interactive_fanfic(message):
    global dialog_check
    bot.reply_to(message, "Какую книгу возьмем за основу фанфика?", reply_markup=create_options_for_interactive_acts())
    dialog_check = 1
    print(f"[DEBUG] Setting dialog_check to 1")

@bot.message_handler(func=lambda message: message.text == "Включить поиск")
def handle_interactive_fanfic(message):
    global dialog_check
    bot.reply_to(message, "Опиши сюжет книги, которую нужно найти :)")
    dialog_check = 2
    print(f"[DEBUG] Setting dialog_check to 2")

@bot.message_handler(func=lambda message: message.text in fandoms)
def handle_fandom(message):
    global context_q
    global dialog_check
    global df
    user_id = message.from_user.id
    fandom = message.text

    # Получаем промпт для выбранного фандома
    prompt = get_fandom_prompt(df, fandom)

    if dialog_check == 1:
      answer, context_q = process_dialogue_step(prompt, context_q)
    if answer is not None:
        bot.reply_to(message, f"{answer}\nПродолжать?", reply_markup=create_picture_keyboard())

@bot.message_handler(func=lambda message: True)
def answer_the_question(message):
    global context_q
    global search_result
    global dialog_check
    print(f"[DEBUG] dialog_check={dialog_check}, context_q={len(context_q)}")
    user_message = message.text

    # Проверка на команду завершения
    if user_message.lower() in ["стоп", "хватит", "закончим"]:
        bot.reply_to(message, "Хорошо, давай закончим. Если захочешь ещё историй, просто напиши!")
        context_q.clear()  # Очищаем контекст
        dialog_check = 0
        return

    if dialog_check == 0:
        bot.reply_to(message, "Привет! Я Элфо! Что почитаем сегодня?", reply_markup=create_keyboard())
        #dialog_check = 1
        return

    # if user_message == "Интерактивный фанфик":
    #     dialog_check = 1
    #     return
    # if user_message == "📚 Выбрать фандом":
    #     dialog_check = 2

    if dialog_check == 2:
        query = user_message
        print(f"[DEBUG] query={query}")
        search_result = find_similar(query, chunks_litres, vectors_litres, model, top_k=1)
        print(f"[DEBUG] search_result={search_result}")
        dialog_check = 1
        print(f"[DEBUG] dialog_check={dialog_check}")


    if dialog_check == 1:
        book_prompt = f'За основу фанфика возьмем: {user_message}' if not context_q else user_message
        print(f"[DEBUG] book_prompt={book_prompt}")
        additional_prompt = ""
        if search_result:
          additional_prompt = search_result
        print(f"[DEBUG] additional_prompt={additional_prompt}")
        answer, context_q = process_dialogue_step(book_prompt, context_q, additional_prompt)
        #answer, context_q = process_dialogue_step(book_prompt, context_q, search_res=search_result if search_result else None)

        if answer is not None:
            bot.reply_to(message, answer, reply_markup=create_keyboard())
            bot.reply_to(message, "Продолжаем?")
            search_result = ""
    return

In [ ]:
dialog_check = 0
context_q = []
answer = None
search_result = ""

In [ ]:
bot.polling(none_stop=True)

[DEBUG] dialog_check=0, context_q=0
[DEBUG] Setting dialog_check to 1
[DEBUG] Setting dialog_check to 2
[DEBUG] dialog_check=2, context_q=0
[DEBUG] query=великан
[DEBUG] search_result=['Название: Очарованный странник, Описание: «очарованный странник» – одно из лучших произведений выдающегося русского писателя николая семёновича лескова (1831–1895). в повести органично соединены жанровые особенности жития, былины, приключенческого романа и романа путешествий. главный герой, простолюдин иван северьянович флягин, странствует по свету, попадает в самые неожиданные ситуации, сталкивается с разными людьми. он принимает жизнь как дар, как великое чудо, у которого нет границ. куда бы ни забросила судьба героя, он открывает для себя нечто новое, удивительное и, возможно, потому нисколько не страшится перемен. приспосабливаясь к обстоятельствам, он меняет род занятий, а порой даже имя. пройдя путь от гpexa к покаянию, к искуплению вины, иван фрязин уходит в монастырь, считая, что так предопредел

In [ ]:
def send_message_with_picture_keyboard(chat_id, text):
    bot.send_message(chat_id=chat_id, text=text, reply_markup=create_picture_keyboard())

In [ ]:
from telebot import types
from collections import deque
import logging
import random


# Инициализация контекста
MAX_CONTEXT_LENGTH = 5
user_contexts = {}
dialog_check = 0

# Создаем клавиатуру
def create_keyboard():
    keyboard = types.ReplyKeyboardMarkup(row_width=2, resize_keyboard=True)
    button_random = types.KeyboardButton("🎲 Рандомный сюжет от Элфо")
    button_fandom = types.KeyboardButton("📚 Выбрать фандом")
    button_continue = types.KeyboardButton("➡️ Продолжить историю")
    keyboard.add(button_random, button_fandom, button_continue)
    return keyboard

# Клавиатура для выбора фандома
def create_fandom_keyboard():
    keyboard = types.ReplyKeyboardMarkup(row_width=2, resize_keyboard=True)
    button_hp = types.KeyboardButton("🪄 Гарри Поттер")
    button_lotr = types.KeyboardButton("🧙 Властелин Колец")
    button_sw = types.KeyboardButton("⭐ Звёздные Войны")
    keyboard.add(button_hp, button_lotr, button_sw)
    return keyboard

# Функции для промптов
def get_random_prompt():
    prompts = [
        "Жил-был дракон, который любил чай...",
        "Однажды в далёкой галактике...",
        "В маленькой деревне жил волшебник, который боялся магии..."
    ]
    return random.choice(prompts)

def get_fandom_prompt(fandom):
    prompts = {
        "🪄 Гарри Поттер": "Напиши историю о Гарри Поттере, который обнаружил новое заклинание...",
        "🧙 Властелин Колец": "Напиши историю о Фродо, который нашёл новое кольцо...",
        "⭐ Звёздные Войны": "Напиши историю о Люке Скайуокере, который встретил нового дроида..."
    }
    return prompts.get(fandom, "Фандом не найден.")

# Управление контекстом
def get_user_context(user_id):
    if user_id not in user_contexts:
        user_contexts[user_id] = deque(maxlen=MAX_CONTEXT_LENGTH)
    return user_contexts[user_id]

# Обработка нажатий на кнопки
@bot.message_handler(func=lambda message: message.text == "🎲 Рандомный сюжет от Элфо")
def random_story(message):
    prompt = get_random_prompt()
    bot.reply_to(message, prompt)

@bot.message_handler(func=lambda message: message.text == "📚 Выбрать фандом")
def choose_fandom(message):
    bot.reply_to(message, "Выбери фандом:", reply_markup=create_fandom_keyboard())

@bot.message_handler(func=lambda message: message.text in ["🪄 Гарри Поттер", "🧙 Властелин Колец", "⭐ Звёздные Войны"])
def handle_fandom(message):
    user_id = message.from_user.id
    fandom = message.text

    # Получаем промпт для выбранного фандома
    prompt = get_fandom_prompt(fandom)

    # Отправляем запрос к модели
    try:
        answer = dialogue_model(prompt, list(get_user_context(user_id)))
    except Exception as e:
        logging.error(f"Ошибка в dialogue_model: {e}")
        bot.reply_to(message, "Произошла ошибка. Попробуйте ещё раз.")
        return

    if not answer:
        logging.error("Пустой ответ от dialogue_model")
        bot.reply_to(message, "Кажется, Элфо заснул... Попробуйте ещё раз.")
        return

    # Отправляем ответ пользователю
    bot.reply_to(message, answer)

    # Обновляем контекст
    context_q = get_user_context(user_id)
    context_q.append({"user": prompt, "assistant": answer})

    # Возвращаем основную клавиатуру
    bot.reply_to(message, "Что будет дальше?", reply_markup=create_keyboard())

@bot.message_handler(func=lambda message: message.text == "➡️ Продолжить историю")
def continue_story(message):
    user_id = message.from_user.id
    context_q = get_user_context(user_id)
    if context_q:
        bot.reply_to(message, "Продолжи историю:")
    else:
        bot.reply_to(message, "Сначала выбери сюжет или фандом!", reply_markup=create_keyboard())

# Основная логика
@bot.message_handler(func=lambda message: True)
def answer_the_question(message):
    global dialog_check

    user_id = message.from_user.id
    context_q = get_user_context(user_id)

    def process_dialogue_step(user_message, context):
        try:
            answer = dialogue_model(user_message, list(context))
        except Exception as e:
            logging.error(f"Ошибка в dialogue_model: {e}")
            bot.reply_to(message, "Произошла ошибка. Попробуйте ещё раз.")
            return None, None

        if not answer:
            logging.error("Пустой ответ от dialogue_model")
            bot.reply_to(message, "Кажется, Элфо заснул... Попробуйте ещё раз.")
            return None, None

        bot.reply_to(message, answer)
        context.append({"user": user_message, "assistant": answer})
        logging.info(f"Updated context: {context}")

        return answer, context

    user_message = message.text

    if user_message.lower() in ["очистить", "стоп", "хватит"]:
        context_q.clear()
        dialog_check = 0
        bot.reply_to(message, "Контекст очищен. Начнём с чистого листа!", reply_markup=create_keyboard())
        return

    if dialog_check == 0:
        bot.reply_to(message, "Привет! Я Элфо! Какую историю ты хочешь почитать?", reply_markup=create_keyboard())
        dialog_check = 1
        return

    if dialog_check == 1:
        answer, context_q = process_dialogue_step(user_message, context_q)

        if answer is not None:
            bot.reply_to(message, answer)
            bot.reply_to(message, "Что будет дальше?", reply_markup=create_keyboard())
        return

# Запуск бота
bot.polling()

In [ ]:
bot.polling(none_stop=True)

In [ ]:



# # @bot.message_handler(commands=['start'])
# # def greet_user(message):
# #   bot.send_message(message.chat.id, "")

# import logging
# logging.basicConfig(level=logging.INFO)

# dialog_check = 0
# context_q = []
# question = ""
# @bot.message_handler(func=lambda message: True)
# def answer_the_question(message):

#   global dialog_check
#   global context_q
#   global question

#   if dialog_check == 0:
#     bot.reply_to(message, "Привет! Я Элфо! Какую историю, ты хочешь почитать?")
#     dialog_check = 1
#     return

#   if dialog_check == 1:
#     user_message = message.text
#     try:
#       answer = dialogue_model(user_message, context_q)
#     except Exception as e:
#       logging.error(f"Ошибка в dialogue_model: {e}")
#       bot.reply_to(message, "Произошла ошибка. Попробуйте ещё раз.")
#       return


#     bot.reply_to(message, answer)

#     if not answer:
#       logging.error("Пустой ответ от dialogue_model")
#       bot.reply_to(message, "Кажется, Элфо заснул... Попробуйте ещё раз.")
#       return

#     context_q = update_context(context_q, user_message, answer)

#     logging.info(f"Updated context: {context_q}")
#     dialog_check = 2
#     return

#   if dialog_check == 2:
#     user_message = message.text
#     answer = dialogue_model(user_message, context_q)
#     bot.reply_to(message, answer)
#     ## обновляем контекст
#     context_q = update_context(context_q, user_message, answer)
#     print(context_q)
#     dialog_check = 0
#     return

In [ ]:
## вот тут все хорошо работало:

# @bot.message_handler(func=lambda message: True)
# def answer_the_question(message):
#     global context_q
#     global dialog_check

#     def process_dialogue_step(user_message, context):
#         try:
#             answer = dialogue_model(user_message, list(context))  # Преобразуем deque в список
#         except Exception as e:
#             logging.error(f"Ошибка в dialogue_model: {e}")
#             bot.reply_to(message, "Произошла ошибка. Попробуйте ещё раз.")
#             return None, None

#         if not answer:
#             logging.error("Пустой ответ от dialogue_model")
#             bot.reply_to(message, "Кажется, Элфо заснул... Попробуйте ещё раз.")
#             return None, None

#         bot.reply_to(message, answer)
#         context = update_context(context, user_message, answer)
#         logging.info(f"Updated context: {context}")

#         return answer, context

#     user_message = message.text

#     # Проверка на команду завершения
#     if user_message.lower() in ["стоп", "хватит", "закончим"]:
#         bot.reply_to(message, "Хорошо, давай закончим. Если захочешь ещё историй, просто напиши!")
#         context_q.clear()  # Очищаем контекст
#         return

#     if dialog_check == 0:
#         bot.reply_to(message, "Привет! Я Элфо! Какую историю ты хочешь почитать?")
#         dialog_check = 1
#         return

#     # Обработка шага диалога
#     if dialog_check == 1:
#       answer, context_q = process_dialogue_step(user_message, context_q)


#     if context_q:  # Если ответ успешно сгенерирован
#         bot.reply_to(message, "Что будет дальше?😜")  # Подсказка для продолжения

#     return